In [7]:
import tensorflow as tf
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('./MNIST_data/',one_hot=True,reshape=False) #载入MNIST数据集，如果指定地址下没有下载好的数据，那么TensorFlow会自动在网站上下载数据

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


# 搭建函数
* 设置参数（lr，epoch，batch，n_hidden_layer,input_size,n_classes)
* 定义参数

In [9]:
# 设置参数
lr = 0.05
epochs = 10
batch = 128
n_hidden_layer = 256

input_size = 784
n_classes = 10

In [22]:
# 设置权重
weights = {
    'hidden_layer':tf.Variable(tf.random_normal([input_size,n_hidden_layer])),
    'out':tf.Variable(tf.random_normal([n_hidden_layer,n_classes]))
}
biases = {
    'hidden_layer':tf.Variable(tf.random_normal([n_hidden_layer])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [23]:
# 设置占位符
x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])

x_flat = tf.reshape(x,[-1,input_size])

In [24]:
# 搭建模型：线性+relu--》前向传播

layer1 = tf.add(tf.matmul(x_flat,weights['hidden_layer']),biases['hidden_layer'])
relu_out = tf.nn.relu(layer1)
layer2 = tf.add(tf.matmul(relu_out,weights['out']),biases['out'])


In [25]:
# 设置优化器 --》反向传播
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=layer2,labels=y))
optim = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

In [26]:
# save 
save_path = './MNIST_data/twoLayerModel.ckpt'
saver = tf.train.Saver() # 用于存储variable，在会话关闭前使用

In [27]:
# 设置会话，利用Mini-batch搭建
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        batch_size = int(data.train.num_examples/batch)
        for i in range(batch_size):
            batch_x,batch_y = data.train.next_batch(batch)
            sess.run(optim,feed_dict={x:batch_x,y:batch_y})
    saver.save(sess,save_path)

In [29]:
with tf.Session() as sess:
    saver.restore(sess,save_path)
    print('weights:\n')
    print(sess.run(weights['out']))
    print('bias\n')
    print(sess.run(biases['out']))

INFO:tensorflow:Restoring parameters from ./MNIST_data/twoLayerModel.ckpt
weights:

[[-0.1087328  -0.5509199   0.60655576 ...  0.81665146 -0.6265526
  -0.02853541]
 [-0.05089313 -0.27206516 -0.09355816 ... -0.54513437 -0.01447545
   1.3493054 ]
 [ 0.99768186 -1.2211367  -0.04995759 ...  0.2425555   0.01890043
   0.6106119 ]
 ...
 [-1.0128969   0.5856892   1.0851734  ...  1.3783652   1.0268177
  -0.5786132 ]
 [-1.4061098  -1.6357385  -0.39493576 ...  0.0634276  -0.5166831
  -0.23630835]
 [-1.1995931   0.57059574 -1.047938   ...  0.47136942 -0.28206798
   0.31887034]]
bias

[-2.3535817   0.836574   -0.28394     0.9501009   0.2184424  -0.8735214
 -0.36764616 -1.0580937  -1.1100044   1.6602883 ]
